# Feature extraction for second groundtruth proposal

In [ ]:
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
import spectral.io.envi as envi
import os
import open3d as o3d

from collections import defaultdict
from skimage.filters import gabor_kernel
from scipy import ndimage as ndi
from scipy.stats import gaussian_kde
from spectral import principal_components
from math import sqrt
from sklearn.decomposition import IncrementalPCA

CHUNK_SIZE = 10

With the second groundtruth proposal, the feature extraction process has to be slightly modified. Now, since each chunk of the tree is considered as an independent observational unit (due to the pole damage being more or less important depending on its vertical location), this process must also be applied to the pictures and to the LiDAR and hyperspectral scans.

## RGB - chunkified features

For the RGB images, the first step is dividing the regions with pole into 10 different chunks along the vertical axis, and then extract the histograms as usual.

### Gray histograms

Firstly, the gray histograms can be extracted from the poles without much trouble. However, the vertical borders of the crops have been further cropped by 400 pixels each, to remove artifacts that were present in the plots.

In [ ]:
# Utility functions
def resize_array(arr, final_size):
    return cv2.resize(np.array([arr], dtype=np.float32), dsize=(final_size,1))[0]

def crop(image, vertical_offsets=[0,0]):
    y_nonzero, x_nonzero = np.nonzero(image)
    return image[np.min(y_nonzero) + vertical_offsets[0]:np.max(y_nonzero) + vertical_offsets[1], np.min(x_nonzero):np.max(x_nonzero)]

def crop_color(image, vertical_offsets=[0,0]):
    y_nonzero, x_nonzero, _ = np.nonzero(image)
    return image[np.min(y_nonzero) + vertical_offsets[0]:np.max(y_nonzero) + vertical_offsets[1], np.min(x_nonzero):np.max(x_nonzero),:]

In [ ]:
def visualize_histograms_gray():
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = [0,30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    result_dict = {
        'pole_id': [],
        'height_id': [],
        'gray_hist_mean': [],
        'gray_hist_std': []
    }

    for pole_idx, pole_id in enumerate(pole_ids):
        intensity_values = []
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../../RGB/sam_crops/{pole_id}_{rotation}_masked.jpg')
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = crop(img, [400,-400])
            intensity_along_vertical = np.sum(img, axis=1) / np.sum(img != 0, axis=1)
            
            # Plot. just for visualization purposes
            axes[pole_idx, rotation_idx*2].plot(range(len(intensity_along_vertical)), intensity_along_vertical, color='black')
            axes[pole_idx, rotation_idx*2].set_ylim([0,128])
            axes[pole_idx, rotation_idx*2+1].imshow(img, cmap='gray')
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])
            
            # Resize intensity curve and add to intensity values matrix
            intensity_resized = resize_array(intensity_along_vertical, 2000)
            intensity_values.append(intensity_resized)
        
        # Compute mean and stdev on each chunk
        intensity_values = np.array(intensity_values)
        chunks = np.array_split(intensity_values, CHUNK_SIZE, axis=1)
        means, stdevs = [np.mean(chunk) for chunk in chunks] , [np.std(chunk) for chunk in chunks]
        pole_id_dict = 0 if pole_id=='Ny' else int(pole_id)
        result_dict['pole_id'] += ([pole_id_dict] * CHUNK_SIZE)
        result_dict['height_id'] += list(range(CHUNK_SIZE))
        result_dict['gray_hist_mean'] += means
        result_dict['gray_hist_std'] += stdevs
        
    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]
    
    return result_dict

In [ ]:
gray_hist_data = visualize_histograms_gray()
df_gray = pd.DataFrame(gray_hist_data)
df_gray.to_csv(f'../Features/second_proposed_gt/gray_histograms.csv', index=False)

### Color histograms

For the color histograms, the procedure is basically the same.

In [ ]:
def visualize_histograms_color():
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = [0,30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    result_dict = {
        'pole_id': [],
        'height_id': [],
        'red_hist_mean': [],
        'red_hist_std': [],
        'green_hist_mean': [],
        'green_hist_std': [],
        'blue_hist_mean': [],
        'blue_hist_std': [],
    }

    for pole_idx, pole_id in enumerate(pole_ids):
        intensity_values_r = []
        intensity_values_g = []
        intensity_values_b = []
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../../RGB/sam_crops/{pole_id}_{rotation}_masked.jpg')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = crop_color(img, [400,-400])
            intensity_along_vertical = np.sum(img, axis=1) / np.sum(img != 0, axis=1)
            
            # Plot. just for visualization purposes
            axes[pole_idx, rotation_idx*2].plot(range(len(intensity_along_vertical[:,0])), intensity_along_vertical[:,0], color='red')
            axes[pole_idx, rotation_idx*2].plot(range(len(intensity_along_vertical[:,1])), intensity_along_vertical[:,1], color='green')
            axes[pole_idx, rotation_idx*2].plot(range(len(intensity_along_vertical[:,2])), intensity_along_vertical[:,2], color='blue')
            axes[pole_idx, rotation_idx*2].set_ylim([0,128])
            axes[pole_idx, rotation_idx*2+1].imshow(img, cmap='gray')
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])
            
            # Resize intensity curve and add to intensity values matrix
            intensity_values_r.append(resize_array(intensity_along_vertical[:,0], 2000))
            intensity_values_g.append(resize_array(intensity_along_vertical[:,1], 2000))
            intensity_values_b.append(resize_array(intensity_along_vertical[:,2], 2000))
        
        # Compute mean and stdev on each chunk
        intensity_values_r, intensity_values_g, intensity_values_b = np.array(intensity_values_r), np.array(intensity_values_g), np.array(intensity_values_b)
        chunks_r = np.array_split(intensity_values_r, CHUNK_SIZE, axis=1)
        chunks_g = np.array_split(intensity_values_g, CHUNK_SIZE, axis=1)
        chunks_b = np.array_split(intensity_values_b, CHUNK_SIZE, axis=1)
        means_r, stdevs_r = [np.mean(chunk) for chunk in chunks_r] , [np.std(chunk) for chunk in chunks_r]
        means_g, stdevs_g = [np.mean(chunk) for chunk in chunks_g] , [np.std(chunk) for chunk in chunks_g]
        means_b, stdevs_b = [np.mean(chunk) for chunk in chunks_b] , [np.std(chunk) for chunk in chunks_b]
        pole_id_dict = 0 if pole_id=='Ny' else int(pole_id)
        result_dict['pole_id'] += ([pole_id_dict] * CHUNK_SIZE)
        result_dict['height_id'] += list(range(CHUNK_SIZE))
        result_dict['red_hist_mean'] += means_r
        result_dict['red_hist_std'] += stdevs_r
        result_dict['green_hist_mean'] += means_g
        result_dict['green_hist_std'] += stdevs_g
        result_dict['blue_hist_mean'] += means_b
        result_dict['blue_hist_std'] += stdevs_b
        
    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]
    
    return result_dict

In [ ]:
color_hist_data = visualize_histograms_color()
df_color = pd.DataFrame(color_hist_data)
df_color.to_csv(f'../Features/second_proposed_gt/color_histograms.csv', index=False)

### Gabor filters

The procedure for obtaining the chunkified Gabor filters features will be similar to what has been done so far.

In [ ]:
# Skimage version 0.19.3
# OpenCV version 4.6.0

kernel_params = [
    ('skimage', (0, 12, 0.01, 0)),
    ('skimage', (0, 12, 0.05, 0)),
    ('skimage', (0, 12, 0.25, 0)),
    ('skimage', (1, 12, 0.01, 0)),
    ('skimage', (1, 12, 0.05, 0)),
    ('skimage', (1, 12, 0.25, 0)),
    ('skimage', (2, 12, 0.01, 0)),
    ('skimage', (2, 12, 0.05, 0)),
    ('skimage', (2, 12, 0.25, 0)),
    ('skimage', (3, 12, 0.01, 0)),
    ('skimage', (3, 12, 0.05, 0)),
    ('skimage', (3, 12, 0.25, 0)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 2, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 4, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 8, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 2, True)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 4, True)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 8, True)),
]

def power(image, kernel):
        # Normalize images for better comparison.
        image = (image - image.mean()) / image.std()
        return np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 + ndi.convolve(image, np.imag(kernel), mode='wrap')**2)

def plot_activations_kernel_pole_rotation(backend, params, kernel_id, draw=True):
    if draw:
        fig, axes = plt.subplots(5, 8, figsize=(20,16))
    
    # Generate Gabor kernel depending on specified backend
    if backend == 'skimage':
        theta = params[0] / 4. * np.pi
        kernel = gabor_kernel(params[2], theta=theta, sigma_x=params[1], sigma_y=params[1], offset=params[3])
    elif backend == 'cv2':
        kernel_params, inv_scale, transpose = params[:-2], params[-2], params[-1]
        kernel = cv2.getGaborKernel(*kernel_params)
        if transpose:
            kernel = kernel.T
            
    # Compute kernel statistics for all pole figures
    pole_ids = [0,30,41,6,5]
    rotations = [0,90,180,270]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    # Prepare dictionaries to store features
    result_dict = {
        'pole_id': [],
        'height_id': [],
        f'kernel_{kernel_id}_mean': [],
        f'kernel_{kernel_id}_std': [],
        f'kernel_{kernel_id}_max': [],
        f'kernel_{kernel_id}_min': [],
    }
    
    for pole_idx, pole in enumerate(pole_ids):
        activation_curves = []
        for rotation_idx, rotation in enumerate(rotations):
            # Read, crop, and rescale image
            img = cv2.imread(f'../../RGB/sam_crops/{pole}_{rotation}_masked.jpg')
            mask = cv2.imread(f'../../RGB/sam_crops/{pole}_{rotation}_mask.jpg')[:,:,0]
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.blur(img, (10,10))
            img = crop(img)
            mask = crop(mask)
            scale_percent = 15
            width = int(img.shape[1] * scale_percent / 100)
            height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            mask = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
            
            # Properly rescale kernel if it is from OpenCV
            if backend == 'cv2':
                kernel = imutils.resize(kernel, width = width // inv_scale)       
            
            activations = np.real(power(img, kernel))
            activations_masked = np.where(mask == 255, activations, 0)
            remapped_activations = np.interp(activations_masked, (activations_masked.min(), activations_masked.max()), (0, +1))
            activation_along_vertical = np.mean(remapped_activations, axis=1)
            activation_along_vertical = resize_array(activation_along_vertical, 2000)
            
            axes[pole_idx, rotation_idx*2].plot(range(len(activation_along_vertical)), activation_along_vertical)
            axes[pole_idx, rotation_idx*2].set_xticks([])
            axes[pole_idx, rotation_idx*2].set_ylim([-0.1,1.1])
            axes[pole_idx, rotation_idx*2+1].imshow(activations_masked)
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])

            # Print mean and standard deviation to compare plots
            activation_curves.append(activation_along_vertical)
        activation_curves = np.array(activation_curves)
        chunks = np.array_split(activation_curves, CHUNK_SIZE, axis=1)
        means, stdevs, c_max, c_min = [np.mean(chunk) for chunk in chunks] , [np.std(chunk) for chunk in chunks] , [np.max(chunk) for chunk in chunks] , [np.min(chunk) for chunk in chunks]
        pole_id_dict = 0 if pole=='Ny' else int(pole)
        result_dict['pole_id'] += ([pole_id_dict] * CHUNK_SIZE)
        result_dict['height_id'] += list(range(CHUNK_SIZE))
        result_dict[f'kernel_{kernel_id}_mean'] += means
        result_dict[f'kernel_{kernel_id}_std'] += stdevs
        result_dict[f'kernel_{kernel_id}_max'] += c_max
        result_dict[f'kernel_{kernel_id}_min'] += c_min
                
    fig.suptitle('Gabor filter activation histograms')
    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]
    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]
    plt.savefig(f'../Figures/Gabor/boxplots/kernel_{kernel_id}_boxplots.png')
    plt.close()
    fig, axes = plt.subplots()
    axes.imshow(np.real(kernel), cmap='gray')
    axes.set_yticks([])
    axes.set_xticks([])
    plt.savefig(f'../Figures/Gabor/boxplots/kernel_{kernel_id}.png')
    
    return result_dict

In [ ]:
with plt.ioff():
    for kernel_id, params in enumerate(kernel_params):
        data = plot_activations_kernel_pole_rotation(*params, kernel_id)
        df_kernel = pd.DataFrame(data)
        df_kernel.to_csv(f'../Features/second_proposed_gt/gabor_kernel_{kernel_id}.csv', index=False)

## Hyperspectral data

Just as it has been done for the RGB data, a similar process can be applied to the hyperspectral data. After all, what is a hyperspectral scan but an RGB image with hundreds of channels?

In [ ]:
def mask_scan(img, mask):
    img = np.where(img > 0.025, 0, img)
    img = np.where(mask[:,:,0] > 0, np.transpose(img, [2,0,1]), 0)
    img = np.transpose(img, [1,2,0])
    
    return img

def get_pca_model():
    pole_ids = [0,30,41,6,5]
    pca = IncrementalPCA(n_components=10, batch_size=1)
    for pole_idx, pole_id in enumerate(pole_ids):
        paths = os.listdir(os.path.join("C:\\Users\\ext-lugo\\Hyperspectral\\Radiance", str(pole_id)))
        paths = [path for path in paths if "float32.hdr" in path]
        paths.sort()
        for scan_idx in range(len(paths)):
            print(f"Pole {pole_id}, scan number {scan_idx}")
            img = envi.open(os.path.join("C:\\Users\\ext-lugo\\Hyperspectral\\Radiance", str(pole_id), paths[scan_idx])).load()
            mask = cv2.imread(os.path.join("../../Hyperspectral_masks/", f"{str(pole_id)}_{str(scan_idx)}_mask.jpg"))
            white = img[1800:2200,1000:1100,:]
            refl_img = img / np.mean(white, axis=(0,1))
            wls = np.asarray(img.metadata['wavelength'], float)
            img_feat = mask_scan(refl_img, mask)
            
            pca.partial_fit(np.reshape(img_feat, (img_feat.shape[0] * img_feat.shape[1], img_feat.shape[2])))
            
    return pca

def view_hyperspectral_chemical(pca):
    fig, axes = plt.subplots(5, 6, figsize=(40,12))
    pole_ids = [0,30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    result_dict = {
        'pole_id': [],
        'height_id': [],
        f'hyper_pca_0_mean': [],
        f'hyper_pca_0_std': [],
        f'hyper_pca_1_mean': [],
        f'hyper_pca_1_std': [],
        f'hyper_pca_2_mean': [],
        f'hyper_pca_2_std': [],
    }
    
    for pole_idx, pole_id in enumerate(pole_ids):
        paths = os.listdir(os.path.join("C:\\Users\\ext-lugo\\Hyperspectral\\Radiance", str(pole_id)))
        paths = [path for path in paths if "float32.hdr" in path]
        paths.sort()
        activation_accumulator_0 = []
        activation_accumulator_1 = []
        activation_accumulator_2 = []
        for scan_idx in range(len(paths)):
            print(f"Pole {pole_id}, scan number {scan_idx}")
            img = envi.open(os.path.join("C:\\Users\\ext-lugo\\Hyperspectral\\Radiance", str(pole_id), paths[scan_idx])).load()
            mask = cv2.imread(os.path.join("../../Hyperspectral_masks/", f"{str(pole_id)}_{str(scan_idx)}_mask.jpg"))
            white = img[1800:2200,1000:1100,:]
            refl_img = img / np.mean(white, axis=(0,1))
            wls = np.asarray(img.metadata['wavelength'], float)
            refl_img = mask_scan(refl_img, mask)
            
            img_feat = np.reshape(pca.transform(np.reshape(refl_img, (refl_img.shape[0] * refl_img.shape[1], refl_img.shape[2]))), (refl_img.shape[0], refl_img.shape[1], 10))
            activation_along_vertical = np.mean(img_feat, axis=1)
            activation_accumulator_0.append(resize_array(activation_along_vertical[:,0], 2000))
            activation_accumulator_1.append(resize_array(activation_along_vertical[:,1], 2000))
            activation_accumulator_2.append(resize_array(activation_along_vertical[:,2], 2000))
            img_feat[:,:,0] = np.interp(img_feat[:,:,0], (np.min(img_feat[:,:,0]), np.max(img_feat[:,:,0])), (0,1))
            img_feat[:,:,1] = np.interp(img_feat[:,:,1], (np.min(img_feat[:,:,1]), np.max(img_feat[:,:,1])), (0,1))
            img_feat[:,:,2] = np.interp(img_feat[:,:,2], (np.min(img_feat[:,:,2]), np.max(img_feat[:,:,2])), (0,1))
            axes[pole_idx, scan_idx].imshow(img_feat[...,3], vmin=np.min(img_feat), vmax=np.max(img_feat))
            
        activation_accumulator_0, activation_accumulator_1, activation_accumulator_2 = np.array(activation_accumulator_0), np.array(activation_accumulator_1), np.array(activation_accumulator_2)
        chunks_0, chunks_1, chunks_2 = np.array_split(activation_accumulator_0, CHUNK_SIZE, axis=1), np.array_split(activation_accumulator_1, CHUNK_SIZE, axis=1), np.array_split(activation_accumulator_2, CHUNK_SIZE, axis=1)
        means_0, stdevs_0 = [np.mean(chunk) for chunk in chunks_0] , [np.std(chunk) for chunk in chunks_0]
        means_1, stdevs_1 = [np.mean(chunk) for chunk in chunks_1] , [np.std(chunk) for chunk in chunks_1]
        means_2, stdevs_2 = [np.mean(chunk) for chunk in chunks_2] , [np.std(chunk) for chunk in chunks_2]
        pole_id_dict = 0 if pole_id=='Ny' else int(pole_id)
        result_dict['pole_id'] += ([pole_id_dict] * CHUNK_SIZE)
        result_dict['height_id'] += list(range(CHUNK_SIZE))
        result_dict[f'hyper_pca_0_mean'] += means_0
        result_dict[f'hyper_pca_0_std'] += stdevs_0
        result_dict[f'hyper_pca_1_mean'] += means_1
        result_dict[f'hyper_pca_1_std'] += stdevs_1
        result_dict[f'hyper_pca_2_mean'] += means_2
        result_dict[f'hyper_pca_2_std'] += stdevs_2
        
    fig.suptitle('Hyperspectral chemical variation and PCA')

    [axes[0,i].set_title(f'Image #{i}') for i in range(6)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]
    
    return result_dict

In [ ]:
pca = get_pca_model()

In [ ]:
data_hyper = view_hyperspectral_chemical(pca)
df_hyper = pd.DataFrame(data_hyper)
df_hyper.to_csv(f'../Features/second_proposed_gt/hyperspectral.csv', index=False)

## LiDAR scans

Now that the new LiDAR scans of all poles are available, features can also be extracted from these scans. More specifically, an accurate diameter estimate can be given for each chunk of the pole, and this estimate seems to be sensitive to certain pole surface characteristics, such as rot or defects due to cracks. 

Even if the LiDAR scan for pole 30 actually gets some points inside its big crack, and this in turn influences the diameter estimate, this is not neccesarily a bad thing for the estimation; even if this issue can influence the estimates, it can do so in a way that actually reflects the fact that the pole is in very bad condition.

In [ ]:
def get_continuous_mode(estimates):
    distribution = gaussian_kde(estimates)
    x_domain = np.linspace(min(estimates), max(estimates), 1000)
    y_pdf = distribution.pdf(x_domain)
    i = np.argmax(y_pdf)
    return x_domain[i]


def estimate_diameter(pcd, height_threshold):
    estimates = []
    for point in pcd.points:
        points_within_threshold = [p for p in pcd.points if p[2] < point[2]+height_threshold and p[2] > point[2]-height_threshold]
        points_distances = [math.dist(point, p) for p in points_within_threshold]
        idx_max_distance = np.argmax(points_distances)
        max_distance = points_distances[idx_max_distance]
        furthest_point_z_diff = abs(points_within_threshold[idx_max_distance][2] - point[2])
        diameter_estimate = sqrt(max_distance**2 - furthest_point_z_diff**2)
        estimates.append(diameter_estimate)
        
    mu, sigma, mode = np.mean(estimates), np.std(estimates), get_continuous_mode(estimates)
    
    return estimates, mu, sigma, mode

def get_lidar_features(height_threshold):
    pole_ids = [0,30,41,6,5]
    scan_names = ['poleny_2_ransac_crop', 'pole30_2_ransac_crop', 'pole41_1_all_ransac_crop', 'pole6_1_ransac_crop', 'pole5_2_ransac_crop']
    result_dict = {
        'pole_id': [],
        'height_id': [],
        f'diameter_mean': [],
        f'diameter_std': [],
        f'diameter_mode': []
    }
    poles_dir = '../../LiDAR/'
    
    for pole_id, scan_path in zip(pole_ids, scan_names):
        pcd = o3d.io.read_point_cloud(os.path.join(poles_dir, f'NEW_LIDAR/{scan_path}.pcd'))
        fig, axes = plt.subplots(ncols=CHUNK_SIZE, figsize=(5*CHUNK_SIZE, 5))
        pcd_array = np.asarray(pcd.points)
        min_height, max_height = np.min(pcd_array[:,2]), np.max(pcd_array[:,2])
        chunk_size = (max_height - min_height) / CHUNK_SIZE
        chunks = [[point for point in pcd_array if ((min_height + chunk_size*i) < point[2] < (min_height + chunk_size*(i+1)))] for i in range(CHUNK_SIZE)]
        mus, sigmas, modes = [], [], []
        for idx, chunk in enumerate(chunks):
            pcd_chunk = o3d.geometry.PointCloud()
            pcd_chunk.points = o3d.utility.Vector3dVector(chunk)
            estimates, mu, sigma, mode = estimate_diameter(pcd_chunk, height_threshold)
            mus.append(mu)
            sigmas.append(sigma)
            modes.append(mode)
            axes[idx].hist(estimates)
            print(f'Diameter estimation for chunk {idx}: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')
        result_dict['pole_id'] += ([pole_id] * CHUNK_SIZE)
        result_dict['height_id'] += list(range(CHUNK_SIZE))
        result_dict['diameter_mean'] += mus
        result_dict['diameter_std'] += sigmas
        result_dict['diameter_mode'] += modes
    return result_dict

In [ ]:
data_lidar = get_lidar_features(0.01)
df_lidar = pd.DataFrame(data_lidar)
df_lidar.to_csv(f'../Features/second_proposed_gt/lidar.csv', index=False)